# High Rotation Rotator/CCW Integration test

This notebook performs a high rotation stress scenario integration test between the Rotator and the Camera Cable Wrap with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew.

In [ ]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

In [ ]:
test_message = "High Rotation Rotator_CCW Integration Test"

In [ ]:
d = salobj.Domain()

In [ ]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")

In [ ]:
await asyncio.gather(rot.start_task,
                     script.start_task)

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

# Track Rotator to +90deg

In [ ]:
await asyncio.sleep(0.1)

print("Track to +90.0 deg for starting position")

tel = rot.tel_Application.get()
cpos = tel.Position
print(cpos)

vel = 0.068
dt = (90 - cpos) / vel
dpos = vel * 0.1
steps = int(dt / 0.1)

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

for i in range(steps):
    pos = cpos + i*dpos
    await rot.cmd_track.set_start(
        angle=pos,
        velocity=vel,
        tai=salobj.current_tai(),
        timeout=10.
    )
    await asyncio.sleep(0.1)
    
await rot.cmd_stop.start(timeout=30.)

# Perform Track from +90deg to -90deg

In [ ]:
await asyncio.sleep(0.1)

print("Track from +90 deg to -90.0 deg at max velocity")

tel = rot.tel_Application.get()
cpos = tel.Position
print(cpos)

vel = 0.068
dt = (90 + cpos) / vel
dpos = vel * 0.1
steps = int(dt / 0.1)

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

for i in range(steps):
    pos = cpos - i*dpos
    await rot.cmd_track.set_start(
        angle=pos,
        velocity=vel,
        tai=salobj.current_tai(),
        timeout=10.
    )
    await asyncio.sleep(0.1)
    
await rot.cmd_stop.start(timeout=30.)

# Move to Zero

In [ ]:
await asyncio.sleep(0.1)

print("Move to 0.0 deg")

await rot.cmd_positionSet.set_start(angle=0.0)

rot.evt_inPosition.flush()

await rot.cmd_move.start(timeout=30.)

inPOS = await rot.evt_inPosition.aget()

print(inPOS.inPosition)

print("Test complete")

script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"END - {test_message}")